In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications



Using TensorFlow backend.


In [2]:
# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

In [3]:
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'w'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'w'),
            bottleneck_features_validation)


In [4]:
def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy'))
    train_labels = np.array(
        [0] * (nb_train_samples / 2) + [1] * (nb_train_samples / 2))

    validation_data = np.load(open('bottleneck_features_validation.npy'))
    validation_labels = np.array(
        [0] * (nb_validation_samples / 2) + [1] * (nb_validation_samples / 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)

In [6]:
save_bottlebeck_features()


Found 2000 images belonging to 2 classes.
Found 802 images belonging to 2 classes.


In [7]:
train_top_model()

Train on 2000 samples, validate on 800 samples
Epoch 1/50
2000/2000 [==============================] - 3s - loss: 0.7693 - acc: 0.7510 - val_loss: 0.2826 - val_acc: 0.8825
Epoch 2/50
2000/2000 [==============================] - 2s - loss: 0.3935 - acc: 0.8440 - val_loss: 0.2804 - val_acc: 0.8825
Epoch 3/50
2000/2000 [==============================] - 2s - loss: 0.3031 - acc: 0.8850 - val_loss: 0.3104 - val_acc: 0.8750
Epoch 4/50
2000/2000 [==============================] - 2s - loss: 0.2850 - acc: 0.8910 - val_loss: 0.2707 - val_acc: 0.8938
Epoch 5/50
2000/2000 [==============================] - 2s - loss: 0.2240 - acc: 0.9190 - val_loss: 0.3157 - val_acc: 0.8888
Epoch 6/50
2000/2000 [==============================] - 2s - loss: 0.2020 - acc: 0.9200 - val_loss: 0.3109 - val_acc: 0.9000
Epoch 7/50
2000/2000 [==============================] - 2s - loss: 0.1998 - acc: 0.9250 - val_loss: 0.3287 - val_acc: 0.9000
Epoch 8/50
2000/2000 [==============================] - 2s - loss: 0.1717 - ac